In [1]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import seaborn as sns
import gc
import random

from gensim.models import Word2Vec

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding
from keras.regularizers import l1, l2

from keras.models import Model, Sequential, load_model
from keras.layers import Input, Flatten, Dense, Conv1D, MaxPooling1D, GlobalMaxPool1D, SpatialDropout1D, \
                          UpSampling1D, LSTM, RepeatVector, TimeDistributed, GRU, Bidirectional, concatenate, \
                          Reshape, Dropout, BatchNormalization, Embedding, GlobalAveragePooling2D, Masking, GRU, \
                          TimeDistributed

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle
from keras.utils import to_categorical, plot_model

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#from utils import get_model, find_closest, get_sequences, create_train_valid,  generate_output, guess_human

import warnings
warnings.filterwarnings('ignore', category = RuntimeWarning)
warnings.filterwarnings('ignore', category = UserWarning)
pd.set_option('max_colwidth', -1)

Using TensorFlow backend.
C:\Users\Enric\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
data = pd.read_csv('data_cleaned_20190514.csv', index_col = None)
print(data.shape)
data.head(3)

(8192, 17)


,Player,Tag,TeamId,Team,DeckId,Archetype,Usage,CardKeysRaw,Games,Losses,Wins,BattleTime,date,CardKeysRawSorted,deck,times,deck_id_int
0,Raider™02,LGG0QU2,55,Black EF / PG / CTP - Jameo eSports,9246338,Hogrider,SameClan,"skeletons,ice-spirit,the-log,valkyrie,poison,electro-wizard,inferno-tower,hog-rider",1,1,0,2019-05-13,2019-05-13,"electro-wizard, hog-rider, ice-spirit, inferno-tower, poison, skeletons, the-log, valkyrie","electro-wizard, hog-rider, ice-spirit, inferno-tower, poison, skeletons, the-log, valkyrie",3,515
1,Kaiju JN,LYU9C09J,56,Black EF / PG / CTP - Kaiju eSports,9246056,Hogrider,Friendly,"skeletons,ice-spirit,the-log,valkyrie,poison,electro-wizard,inferno-tower,hog-rider",1,1,0,2019-05-12,2019-05-12,"electro-wizard, hog-rider, ice-spirit, inferno-tower, poison, skeletons, the-log, valkyrie","electro-wizard, hog-rider, ice-spirit, inferno-tower, poison, skeletons, the-log, valkyrie",3,515
2,MAD MCasc,R99YP00J,74,SLO - MAD Lions E.C.,9246027,Hogrider,SameClan,"skeletons,ice-spirit,the-log,valkyrie,poison,electro-wizard,inferno-tower,hog-rider",1,1,0,2019-05-12,2019-05-12,"electro-wizard, hog-rider, ice-spirit, inferno-tower, poison, skeletons, the-log, valkyrie","electro-wizard, hog-rider, ice-spirit, inferno-tower, poison, skeletons, the-log, valkyrie",3,515


In [3]:
data = data.loc[data.index.repeat(data.Games)].reset_index(drop=True)

In [4]:
data['CardKeysRaw'] = data['CardKeysRaw'].str.replace('-', '')
data.head(3)

,Player,Tag,TeamId,Team,DeckId,Archetype,Usage,CardKeysRaw,Games,Losses,Wins,BattleTime,date,CardKeysRawSorted,deck,times,deck_id_int
0,Raider™02,LGG0QU2,55,Black EF / PG / CTP - Jameo eSports,9246338,Hogrider,SameClan,"skeletons,icespirit,thelog,valkyrie,poison,electrowizard,infernotower,hogrider",1,1,0,2019-05-13,2019-05-13,"electro-wizard, hog-rider, ice-spirit, inferno-tower, poison, skeletons, the-log, valkyrie","electro-wizard, hog-rider, ice-spirit, inferno-tower, poison, skeletons, the-log, valkyrie",3,515
1,Kaiju JN,LYU9C09J,56,Black EF / PG / CTP - Kaiju eSports,9246056,Hogrider,Friendly,"skeletons,icespirit,thelog,valkyrie,poison,electrowizard,infernotower,hogrider",1,1,0,2019-05-12,2019-05-12,"electro-wizard, hog-rider, ice-spirit, inferno-tower, poison, skeletons, the-log, valkyrie","electro-wizard, hog-rider, ice-spirit, inferno-tower, poison, skeletons, the-log, valkyrie",3,515
2,MAD MCasc,R99YP00J,74,SLO - MAD Lions E.C.,9246027,Hogrider,SameClan,"skeletons,icespirit,thelog,valkyrie,poison,electrowizard,infernotower,hogrider",1,1,0,2019-05-12,2019-05-12,"electro-wizard, hog-rider, ice-spirit, inferno-tower, poison, skeletons, the-log, valkyrie","electro-wizard, hog-rider, ice-spirit, inferno-tower, poison, skeletons, the-log, valkyrie",3,515


## Sequences

In [5]:
original_cards = list(data['CardKeysRaw'])
len(original_cards)

68110

In [6]:
original_cards = [x for x in original_cards]
print(original_cards[:5])

['skeletons,icespirit,thelog,valkyrie,poison,electrowizard,infernotower,hogrider', 'skeletons,icespirit,thelog,valkyrie,poison,electrowizard,infernotower,hogrider', 'skeletons,icespirit,thelog,valkyrie,poison,electrowizard,infernotower,hogrider', 'icespirit,barbarianbarrel,wallbreakers,dartgoblin,infernotower,rascals,miner,poison', 'icespirit,barbarianbarrel,wallbreakers,dartgoblin,infernotower,rascals,miner,poison']


In [7]:
def make_sequences(texts, training_length=8, lower=True):
    """Turn a set of texts into sequences of integers"""
    # Create the tokenizer object and train on texts
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)

    # Create look-up dictionaries and reverse look-ups
    word_idx = tokenizer.word_index
    idx_word = tokenizer.index_word
    num_words = len(word_idx)+1
    word_counts = tokenizer.word_counts

    print(f'There are {num_words} unique words.')

    # Convert text to sequences of integers
    sequences = tokenizer.texts_to_sequences(texts)

    # Limit to sequences with more than training length tokens
    seq_lengths = [len(x) for x in sequences]
    over_idx = [i for i, l in enumerate(seq_lengths)]

    new_texts = []
    new_sequences = []

    # Only keep sequences with more than training length tokens
    for i in over_idx:
        new_texts.append(texts[i])
        new_sequences.append(sequences[i])

    training_seq = []
    labels = []

    # Iterate through the sequences of tokens
    for seq in new_sequences:

        # Create multiple training examples from each sequence
        for i in range(2, training_length):
            #print(i, len(seq))
            # Extract the features and label
            extract = seq[0:i]
            labels.append(seq[len(extract)])
                
            while(len(extract) < 8):
                extract.append(0) #No presence of that item

            # Set the features and label
            training_seq.append(extract)
                  

    print(f'There are {len(training_seq)} training sequences.')

    # Return everything needed for setting up the model
    return word_idx, idx_word, num_words, word_counts, new_texts, new_sequences, training_seq, labels

In [8]:
MAX_SEQ_LENGTH = 8
word_idx, idx_word, num_words, word_counts, abstracts, sequences, features, labels = make_sequences(
    original_cards, MAX_SEQ_LENGTH, lower=True)

There are 92 unique words.
There are 408660 training sequences.


In [9]:
for i in range(0, 8):
    print(features[i])
    print(labels[i])

[16, 14, 0, 0, 0, 0, 0, 0]
12
[16, 14, 12, 0, 0, 0, 0, 0]
62
[16, 14, 12, 62, 0, 0, 0, 0]
9
[16, 14, 12, 62, 9, 0, 0, 0]
11
[16, 14, 12, 62, 9, 11, 0, 0]
21
[16, 14, 12, 62, 9, 11, 21, 0]
30
[16, 14, 0, 0, 0, 0, 0, 0]
12
[16, 14, 12, 0, 0, 0, 0, 0]
62


In [10]:
def find_cards(index):
    """Find label corresponding to features for index in training data"""

    # Find features and label
    feats = ' '.join(idx_word[i] for i in features[index])
    answer = idx_word[labels[index]]

    print('Cards:', feats)
    print('\nLabel: ', answer)
    

### Training Data

In [11]:
def create_train_valid(features,
                       labels,
                       num_words,
                       train_fraction=0.9,
                       seq_length = 8):
    """Create training and validation features and labels."""

    # Randomly shuffle features and labels
    features, labels = shuffle(features, labels, random_state=50)

    # Decide on number of samples for training
    train_end = int(train_fraction * len(labels))

    train_features = np.array(features[:train_end])
    valid_features = np.array(features[train_end:])

    train_labels = labels[:train_end]
    valid_labels = labels[train_end:]

    # Convert to arrays
    X_train, X_valid = np.array(train_features), np.array(valid_features)

    # Using int8 for memory savings
    y_train = np.zeros((len(train_labels), num_words), dtype=np.int8)
    y_valid = np.zeros((len(valid_labels), num_words), dtype=np.int8)

    # One hot encoding of labels
    for example_index, word_index in enumerate(train_labels):
        y_train[example_index, word_index] = 1

    for example_index, word_index in enumerate(valid_labels):
        y_valid[example_index, word_index] = 1

    # Memory management
    import gc
    gc.enable()
    del features, labels, train_features, valid_features, train_labels, valid_labels
    gc.collect()

    return X_train, X_valid, y_train, y_valid

In [12]:
X_train, X_valid, y_train, y_valid = create_train_valid(
    features, labels, num_words, train_fraction=0.8)
print(X_train.shape)
print(y_train.shape)

(326928, 8)
(326928, 92)


In [13]:
X_train[1]

array([3, 2, 0, 0, 0, 0, 0, 0])

In [14]:
def check_sizes(gb_min=1):
    for x in globals():
        size = sys.getsizeof(eval(x)) / 1e9
        if size > gb_min:
            print(f'Object: {x:10}\tSize: {size} GB.')


check_sizes(gb_min=0.01)

Object: data      	Size: 0.065609263 GB.
Object: X_train   	Size: 0.010461808 GB.
Object: y_train   	Size: 0.030077488 GB.


## Model Architecture

In [15]:
EMBEDDING_DIM = 20
MAX_SEQ_LENGTH = 8

In [16]:
w2v = Word2Vec.load("Cards_Embedding_v4")
word_vectors = w2v.wv
vocabulary_size = len(word_idx) + 1
embedding_matrix = np.zeros((len(word_idx) + 1, EMBEDDING_DIM))

for word, i in word_idx.items():
    if word in w2v:
        embedding_matrix[i] = w2v[word]
    else:
        embedding_matrix[i] = np.random.rand(1, EMBEDDING_DIM)[0]
            

del(word_vectors)
gc.collect()

C:\Users\Enric\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
C:\Users\Enric\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


0

In [23]:
def get_model_Seq(CAT_OUTPUTS):
    model = Sequential()

    # Embedding layer
    model.add(Embedding(input_dim=num_words,
                    output_dim=EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    mask_zero=True,
                    trainable=False, name = 'Cards'))
    
    #model.add(Masking()) #Para los zeros

    # Recurrent layer
    #model.add(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.1))
    #model.add(Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.1)))
    model.add(Bidirectional(LSTM(64, return_sequences=False, dropout=0.2, recurrent_dropout=0.1)))

    # Fully connected layer
    model.add(Dense(128, activation = 'relu'))

    # Dropout for regularization
    model.add(Dropout(0.4))
    
    model.add(Dense(64, activation = 'relu'))
    
    model.add(Dropout(0.3))

    # Output layer
    model.add(Dense(CAT_OUTPUTS, activation='softmax'))

    # Compile the model
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    model.summary()
    
    return model

In [24]:
model = get_model_Seq(num_words)

history = model.fit(X_train, y_train,
                    epochs=3,
                    batch_size=128,
                    verbose=1,
                    validation_data=(X_valid, y_valid))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Cards (Embedding)            (None, None, 20)          1840      
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               43520     
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 92)                5980      
Total para

In [25]:
model.save('full_model_20190515')
#model = load_model('full_model_20190514')

In [26]:
ev = model.evaluate(X_valid, y_valid, verbose = 1)
print(f'Crossentropy Loss: {ev[0]}')
print(f'Acc: {ev[1]}')

81732/81732 [==============================] - 15s 189us/step
Crossentropy Loss: 0.9092661156759606
Acc: 0.7280746831106544


In [35]:
seq = features[101237]
print(seq)
print([idx_word[i] for i in seq if i!= 0])
print(np.array(seq).argmin())
idx = np.array(seq).argmin()
idx_word[model.predict(np.array(seq).reshape(1, -1))[0].argmax()]

[14, 16, 3, 12, 52, 57, 6, 0]
['icespirit', 'skeletons', 'icegolem', 'thelog', 'archers', 'tesla', 'fireball']
7


'xbow'

In [49]:
#training_length=7
new_words=1
diversity=1
return_output=False
n_gen=1

seq = random.choice(features)
print(f'Original seq: {seq}\n')

seed_idx = np.array(seq).argmin()
end_idx = MAX_SEQ_LENGTH - 1
#print(seed_idx, end_idx)

completed_cards = seed_idx

original_sequence = [idx_word[i] for i in seq[:seed_idx]]
print(f'Original Cards: {original_sequence}')

# Keep adding new cards
while (completed_cards < 8):
    preds = model.predict(np.array(seq).reshape(1, -1))[0]

    print(f'Next Card: {idx_word[preds.argmax()]}')
    seq[seed_idx] = word_idx[idx_word[preds.argmax()]]
    seed_idx += 1
    actual = [idx_word[i] for i in seq[:seed_idx]]
    print(f'Cards: {actual}\n')
    completed_cards += 1


Original seq: [89, 2, 53, 10, 0, 0, 0, 0]

Original Cards: ['mirror', 'giantsnowball', 'speargoblins', 'bandit']
Next Card: darkprince
Cards: ['mirror', 'giantsnowball', 'speargoblins', 'bandit', 'darkprince']

Next Card: minionhorde
Cards: ['mirror', 'giantsnowball', 'speargoblins', 'bandit', 'darkprince', 'minionhorde']

Next Card: elixircollector
Cards: ['mirror', 'giantsnowball', 'speargoblins', 'bandit', 'darkprince', 'minionhorde', 'elixircollector']

Next Card: threemusketeers
Cards: ['mirror', 'giantsnowball', 'speargoblins', 'bandit', 'darkprince', 'minionhorde', 'elixircollector', 'threemusketeers']



In [63]:
def get_next_card(model, features, training_length=8, diversity=1, return_output=False, n_gen=1):
    print('=='*50)
    seq = random.choice(features)
    print(f'Original seq: {seq}')

    seed_idx = np.array(seq).argmin()
    end_idx = training_length - 1
    completed_cards = seed_idx

    original_sequence = [idx_word[i] for i in seq[:seed_idx]]
    print(f'Original Cards: {original_sequence}')
    print('=='*50+'\n')

    # Keep adding new cards
    while (completed_cards < 8):
        preds = model.predict(np.array(seq).reshape(1, -1))[0]

        print(f'Next Card: {idx_word[preds.argmax()]}')
        seq[seed_idx] = word_idx[idx_word[preds.argmax()]]
        seed_idx += 1
        actual = [idx_word[i] for i in seq[:seed_idx]]
        print(f'Cards: {actual}\n')
        completed_cards += 1

        #if return_output:
            #return original_sequence, gen_list, a

In [67]:
get_next_card(model, features, training_length = 8)

Original seq: [1, 42, 37, 0, 0, 0, 0, 0]
Original Cards: ['barbarianbarrel', 'icewizard', 'tombstone']

Next Card: tornado
Cards: ['barbarianbarrel', 'icewizard', 'tombstone', 'tornado']

Next Card: babydragon
Cards: ['barbarianbarrel', 'icewizard', 'tombstone', 'tornado', 'babydragon']

Next Card: darkprince
Cards: ['barbarianbarrel', 'icewizard', 'tombstone', 'tornado', 'babydragon', 'darkprince']

Next Card: poison
Cards: ['barbarianbarrel', 'icewizard', 'tombstone', 'tornado', 'babydragon', 'darkprince', 'poison']

Next Card: graveyard
Cards: ['barbarianbarrel', 'icewizard', 'tombstone', 'tornado', 'babydragon', 'darkprince', 'poison', 'graveyard']

